# <center><span style="font-size: 50px; color: blue;">ARCHITECTURE V1.2 DEMO</span></center>

<center><span style="font-size: 25px; color: purple;">This notebook shows how to use our new architecture to perform a simulation</span></center>
<p></p>

<center><span style="font-size: 16px; color: black;">Please make sure you installed our module named <b><i>filament<i/></b> by running the following commands in a terminal:</span></center>
<p></p>

<center><code style="font-size: 16px; color: green;">$ cd PIE_meteo/arch/
$ python setup.py install

# Our module is named filament

In [ ]:
from filament.core.grid import Grid
from filament.core.state import State
from filament.core.simulation import Simulation
from filament.test.test_cases import v_stripe_test, bubble_test, gaussian_test

# Simulation parameters

In [ ]:
Lx = 2048e3
Ly = 1024e3
Nx = 256
Ny = 128


T = 2*3600 # Complete simulation is no more very long
dt = 300
Nt = int(T//dt)


dX = Nx//8  # used to shape the initial v-stripe data
dY = Ny//15

nb_state = 2  # number of instants in initial data

# We import the scientific methods (Hi Olivier & Max)

In [ ]:
from filament.methods.pseudo_spectral_wind import pseudo_spectral_wind
from filament.methods.wrap_advection_step_3P import wrap_advection_step_3P
from filament.methods.wrap_wv import wrap_wv
from filament.methods.end_pop import end_pop

Your methods have just been wrapped in such a way it fits our architechture, here is an example of a stupid method. Nothing of scientific, but you can see its signature, and basically understand how we expect a method to interact with data.

In [ ]:
def stupid_method(history, verbose, *args, **kwargs):
    last_state = history.state_list[-1]
    
    new_state = State.copy(last_state)
    new_state.t += dt
    
    print(new_state.t) if verbose else None
    
    new_state.vrs['theta_t'][int(new_state.t) % Nx, int(new_state.t) % Ny] = 1 # yes it is stupid
    
    history.append(new_state) # adding the new fresh state
    history.pop(0) # deleting the oldest state

# Simulation parameters and building

In [ ]:
methods = [pseudo_spectral_wind,
           wrap_advection_step_3P,wrap_wv,end_pop]

methods_kwargs = [{},
                  {'alpha_method' : 'damped_bicubic',
                   'order_alpha' : 2, 
                   'F_method' : 'damped_bicubic'},
                  {'alpha_method' : 'damped_bicubic',
                   'order_alpha' : 2, 
                   'F_method' : 'damped_bicubic'},
                  {}]

output_folder = 'output_test'
save_rate = 1
backup_rate = 10
verbose = 1 # displaying level, usefull to inspect what's going wrong

In [ ]:
# Creation of the test case
initialCDF = v_stripe_test('initial.nc', Lx, Ly, Nx, Ny, dt, nb_state, dX, dY)

$\textbf{New !}$
T, Nt, save_rate, backup_rate are no more a Simulation object parameter. You give them when you run the simulation, it allows you to continue the simulation by running again with new parameters linked to the simulation time (the new run will start from the end of the previous one). You'll test it in few cells. 
methods_kwargs are now mandatory (source of bug when it was not) - there is an other option we'll discuss about it. If your methods doesn't except any argument, give a list of void dictionary with the same length as methods list.

In [ ]:
mySim = Simulation(initialCDF,
                   methods, 
                   methods_kwargs,
                   output_folder,
                   verbose=verbose, 
                   name='testfb')

The method run has a last argument (first_run default True). If first_run is True, the first and the last state will be saved (obviously some intermediate states too), if it is not, the first state won't be saved as it correspond to the last saved state (which is then already saved).

In [ ]:
mySim.run(T, Nt, save_rate, backup_rate, first_run=True)

### if you want to extend the simulation a little bit...

We will extend the first run (2h of simulation) with a new one (5h)

In [ ]:
T2 = 5*3600 
Nt2 = int(T2//dt)
save_rate = 2
backup_rate = 5

$\textbf{Warning !}$ For recovery from backup purpose save_rate must divide backup_rate : check the return Exception

In [ ]:
mySim.run(T2, Nt2, save_rate, backup_rate, first_run=False)

### With compatible save/backup rates :

$\textbf{Try interrupting this run ! (interrupt the kernel during simulation)}$ You will see how to launch a new simulation from the backup file to continue the simulation later

In [ ]:
save_rate = 2
backup_rate = 6
mySim.run(T2, Nt2, save_rate, backup_rate, first_run=False)

# netCDF results can easily be analyzed...

In [ ]:
from netCDF4 import Dataset
import numpy as np

In [ ]:
resultsCDF = Dataset('output_test/results_testfb.nc', 'r', format='NETCDF4', parallel=False)
backupCDF = Dataset('output_test/backup_testfb.nc', 'r', format='NETCDF4', parallel=False)

To see the saved times and the last backup time :

In [ ]:
print(resultsCDF['t'][:].data)
print(backupCDF['t'][:].data)

To see the parameters of the different runs :

In [ ]:
print(resultsCDF.T)
print(resultsCDF.Nt)
print(resultsCDF.save_rate)
print(resultsCDF.backup_rate)

Don't forget to close the datasets

In [ ]:
resultsCDF.close()
backupCDF.close()

# <center>...and plotted !

You will create an interactive object linked to a netCDF file (the result one). You can change the plotted variable, time and colormap. Don't worry it is re-opened and re-closed at each change you ask for so the file is not corrupted.

In [ ]:
from filament.display.interacting_plot import interactive_plot

In [ ]:
inter = interactive_plot('output_test/results_testfb.nc')
inter

In [ ]:
from filament.display import animate

In [ ]:
variable = 'Delta_z'

# the CDF file results.nc stored in the output_folder is used to 
# generate a video saved in the output_folder
animate.make_video(output_folder + '/results_testfb.nc', output_folder, variable)

# Checking backup start

If the last run has been interrupted, you'll see how to continue the simulation with the backup file and the result file (to copy the states saved from the beginning to the last backup). If the result file is corrupted or can't be used, you won't retrive the previous data, but if you're only interested in the end of the simulation you can begin the new run at the last backup thanks to this.

In [ ]:
backupCDF = Dataset(output_folder + '/backup_testfb.nc', 'r', format='NETCDF4', parallel=False)
pre_resultCDF = Dataset(output_folder + '/results_testfb.nc', 'r', format='NETCDF4', parallel=False)

In [ ]:
print(backupCDF.methods)

In [ ]:
mySim_fromb = Simulation.frombackup(backupCDF,
                                    methods,
                                    methods_kwargs,
                                    output_folder,
                                    resultCDF=pre_resultCDF,
                                    name='testfb_fb',
                                    verbose=2)

In [ ]:
mySim_fromb.run(#tocomplete,#tocomplete,2,6,first_run=True)

In [ ]:
resultsCDF = Dataset(output_folder + '/results_testfb_fb.nc', 'r', format='NETCDF4', parallel=False)
print(resultsCDF['t'][:].data)
print(resultsCDF.T)
print(resultsCDF.Nt)
print(resultsCDF.save_rate)
print(resultsCDF.backup_rate)
resultsCDF.close()

In [ ]:
previous_resultsCDF = Dataset(output_folder + '/results_testfb.nc', 'r', format='NETCDF4', parallel=False)
print(previous_resultsCDF['t'][:].data)
previous_resultsCDF.close()

## Launching the all simulation in one go to verify the results

In [ ]:
initialCDF = Dataset('initial.nc','r', format='NETCDF4', parallel=False)
verbose=1
mySim = Simulation(initialCDF,
                   methods, 
                   methods_kwargs,
                   output_folder,
                   verbose=verbose,
                   name='control')

In [ ]:
T = 7*3600
dt = 300
Nt = int(T//dt)
save_rate = 1
backup_rate = 10

mySim.run(T, Nt, save_rate, backup_rate, first_run=True)

The two simulation does not have the exact same save_rate (since we changed from 1 to 2 between the first and second part of the first simulation). Keep it in mind while comparing the results

In [ ]:
reference = Dataset(output_folder + '/results_control.nc', 'r', parallel=False)
perturbed = Dataset(output_folder + '/results_testfb_fb.nc', 'r', parallel=False)

In [ ]:
print(reference['t'][:].data)
print(perturbed['t'][:].data)

In [ ]:
k_per = 35
t = perturbed['t'][k_per]
k_ref = np.where(reference['t'][:].data == t)[0][0]

print("t = {}".format(t))
print("k_ref = {}".format(k_ref))

In [ ]:
variable = 'theta_t'
np.min(np.equal(perturbed[variable][:,:,k_per], reference[variable][:,:,k_ref]))

True means that all the values (on the spatial grid) of the variable are equals at instant t

In [ ]:
reference.close()
perturbed.close()